# (0) 필요한 모듈

In [2]:
#!pip install google-api-python-client
#!pip install oauth2client

#YOUTUBE OPEN API 관련 모듈
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser

#기타 모듈
import pandas as pd
import numpy as np
import re #문자열 패턴 관련 모듈
import requests
import os

In [3]:
#현재 working directory 경로 확인
#코드 실행자의 파일 위치에 따라 달라짐
os.getcwd()

'C:\\Users\\lys17\\Desktop\\코드 정리\\ipynb 코드'

**<유튜브 채널>**

1. 하이브(HYBE)
- 소속사 공식 채널: HYBE LABELS
- 소속 아티스트 공식 채널 (5개)
    - NewJeans
    - BANGTANTV
    - LE SSERAFIM
    - SEVENTEEN
    - TOMORROW X TOGETHER OFFICIAL
2. 와이지(YG)
- 소속사 공식 채널: YG ENTERTAINMENT
- 소속 아티스트 공식 채널 (4개)
    - BLACKPINK
    - OfficialGDRAGON
    - TREASURE (트레저)
    - WINNER  
3. 제이와이피(JYP)
- 소속사 공식 채널: JYP Entertainment
- 소속 아티스트 공식 채널 (5개)
    - Stray Kids
    - TWICE
    - NMIXX
    - DAY6
    - ITZY

# (1) 검색결과 가져오기
api에서 제공하는 검색기능을 활용하기 위해 search.list.method를 사용함.
<br>list에 들어가는 중요 파라미터는 다음과 같음.

**<매개변수>**
- q: 검색어
- order: 정렬방식
    - date: 리소스를 만든 날짜를 기준으로 최근 항목부터 시간 순서대로 리소스를 정렬함.
    - rating: 높은 평가부터 낮은 평가순으로 리소스를 정렬함.
    - relevance: 검색 쿼리에 대한 관련성을 기준으로 리소스를 정렬함. (기본값)
    - title: 제목에 따라 문자순으로 리소스를 정렬함.
    - videoCount: 업로드한 동영상 수에 따라 채널을 내림차순으로 정렬함.
    - viewCount: 리소스를 조회수가 높은 항목부터 정렬함.
- part(필수): API 응답이 포함하는 search 리소스 속성을 지정함.
    - 가능한 값: 'id', 'snippet'
    - 매개변수가 하위 속성을 포함하는 속성을 식별할 경우, 하위 속성이 응답에 포함됨. (ex. search 결과에서 snippet 속성은 결과의 제목, 설명 등을 식별하는 다른 속성을 포함함.)
    - "snippet"을 설정하는 경우, API 응답은 중첩된 속성도 모두 포함함.
- maxResults: 최대 호출 개수
    - 값의 범위는 0 ~ 50
- pageToken: 반환해야 하는 결과 집합의 특정 페이지를 식별함
    - API 응답에서 nextPageToken 및 prevPageToken 속성은 검색할 기타 페이지를 식별함.
- publishedAfter: API 응답이 지정된 시간 이후에 만든 리소스만 포함하도록 하는 설정
    - 날짜 형식은 RFC 3339로, 날짜-시간 값(1970-01-01T00:00:00Z)
- publishedBefore: API 응답이 지정된 시간 전에 만든 리소스만 포함하도록 하는 설정
    - 날짜 형식은 RFC 3339로, 날짜-시간 값(1970-01-01T00:00:00Z)
- regionCode: API에서 지정된 국가에 대한 검색결과를 반환하도록 하는 설정
    - 지정된 형식은 ISO 3166-1 alpha-2 국가 코드임. (대한민국: KR)
- type: 특정 리소스 유형만 검색하도록 검색 쿼리를 제한하는 설정
    - 가능한 값: channel, playlist, video
- videoCategoryId: 카테고리를 기준으로 하는 동영상 검색결과를 필터링함
- videoDuration: 동영상 길이를 기준으로 동영상 검색결과를 필터링함
    - 가능한 값: 
        - any: 동영상 길이를 기준으로 동영상 검색결과를 필터링하지 않음. (기본값)
        - long: 20분보다 긴 동영상을 포함함.
        - medium: 4분 이상 20분 이하인 동영상만 포함함.
        - short: 4분 미만인 동영상만 포함함.

<span style = "color:Red;">**!!!주의 사항!!!**</span>

(1) YouTube OPEN API 사용 시, 구글 클라우드 플랫폼에 연결되어 있는 상태인지 확인 필요함. 안 그러면 에러 발생.
<br>=> 구글 클라우드 플랫폼 사이트에서 구글 계정으로 로그인 된 상태에서 '콘솔' 클릭하여 활성화해야 함.

(2) OPEN API로 불러온 데이터는 가능한한 저장을 권장함.
<br>=> 하루 할당량(=약 10,000)이 있기 때문에

In [4]:
#기본 정보 설정
api_key = "***" #api key는 보안상의 문제로 마킹 처리함
youtube_api_service_name = "youtube" #Google API 객체를 생성 (명칭은 'youtube'로 고정됨)
youtube_api_version = "v3"
youtube = build(youtube_api_service_name, youtube_api_version, developerKey=api_key)

## (1-1) 소속사 채널 ID 추출

### (1-1-1) 하이브(HYBE)

In [3]:
#query문 작성
search_response = youtube.search().list(
    q = "HYBE LABELS",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [4]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'zGgMmGbnhB83_OEfdoyJrAZqrjU',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 1435, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'ULm28uTgiw4G74c-7Kgj1rw2_No',
   'id': {'kind': 'youtube#channel', 'channelId': 'UC3IZKseVpdzPSBaWxBxundA'},
   'snippet': {'publishedAt': '2008-06-04T08:23:22Z',
    'channelId': 'UC3IZKseVpdzPSBaWxBxundA',
    'title': 'HYBE LABELS',
    'description': 'Welcome to the official YouTube channel of HYBE, the content hub for BIGHIT MUSIC, BELIFT LAB, SOURCE MUSIC, PLEDIS ...',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AOPolaQdfSRhcE0CNaJkVHMjeKMud1Jqe2D2MrM0kIoHEw=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/ytc/AOPolaQdfSRhcE0CNaJkVHMjeKMud1Jqe2D2MrM0kIoHEw=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/ytc/AOPolaQdfSRhcE0CNaJkVHMjeKMud1Jqe2D2MrM0kIoHEw=s800-c-k-c0

In [33]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,HYBE LABELS,UC3IZKseVpdzPSBaWxBxundA
1,HYBE LABELS +,UCjfqjGIjR3AtVsvzCc4fO1g
2,HYBE LABELS,UCD6TdvomGG2QGHc8SXkVOsQ
3,HYBE LABELS,UCPXorjpHdFAwVhg9Q4tJEYA
4,HYBE LABELS,UCEHPSD3nK-A-CL3yvMcxozA


In [34]:
hybe = search_df["channelId"][0]
hybe

'UC3IZKseVpdzPSBaWxBxundA'

### (1-1-2) 와이지(YG)

In [20]:
#query문 작성
search_response = youtube.search().list(
    q = "YG ENTERTAINMENT",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [21]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': '0JBjfzJS8tuvTOFH0kAi5rKI4DA',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 21792, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': '1fcQqcrL3WMWfsndTwn4rxWq7B0',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCQi67q4kGdmnJaRzX81uK5g'},
   'snippet': {'publishedAt': '2008-01-18T02:22:00Z',
    'channelId': 'UCQi67q4kGdmnJaRzX81uK5g',
    'title': 'YG ENTERTAINMENT',
    'description': 'Welcome to YG Entertainment.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AOPolaTEuA0WE9axm89mn_Wg8n_B19g7YuNyqkas8IgoRQ=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/ytc/AOPolaTEuA0WE9axm89mn_Wg8n_B19g7YuNyqkas8IgoRQ=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/ytc/AOPolaTEuA0WE9axm89mn_Wg8n_B19g7YuNyqkas8IgoRQ=s800-c-k-c0xffffffff-no-rj-mo'}},
    'channelTitle': 'YG ENTERTAINMENT',
    'liveBroadcastCont

In [22]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,YG ENTERTAINMENT,UCQi67q4kGdmnJaRzX81uK5g
1,YG TREASURE BOX,UCquCOGattLleH5erOxiWVmQ
2,YG Entertainment,UCDWzubWCh-wNKN7aa6AGrvQ
3,YG ENTERTAINMENT,UCzXVYRoJlnHY4WspKMh4JTA
4,YG ENTERTAINMENT OFFICIAL,UC2MT3BWUy9RFSvYPc0eftaw


In [23]:
yg = search_df["channelId"][0]
yg

'UCQi67q4kGdmnJaRzX81uK5g'

### (1-1-3) 제이와이피(JYP)

In [4]:
#query문 작성
search_response = youtube.search().list(
    q = "JYP Entertainment",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [5]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'pAzAcM9LCYo9e9tao3f6ybUGmYo',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 12988, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'bTJA2_xt4FjYwdl9fRfSlsHKtnw',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCaO6TYtlC8U5ttz62hTrZgg'},
   'snippet': {'publishedAt': '2008-01-25T07:45:44Z',
    'channelId': 'UCaO6TYtlC8U5ttz62hTrZgg',
    'title': 'JYP Entertainment',
    'description': 'JYPnation Official YouTube LEADER IN ENTERTAINMENT There are values and goals that JYP Entertainment (JYP) has ...',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/kcV7NQkBm-UvvzVTJvrg1Yf1eHSqi-DLXuZPt_ECa3cHEPefujS951Dxj6KUEQ5i9Z7_fyMUjw=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/kcV7NQkBm-UvvzVTJvrg1Yf1eHSqi-DLXuZPt_ECa3cHEPefujS951Dxj6KUEQ5i9Z7_fyMUjw=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/kcV7NQkBm-Uv

In [6]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,JYP Entertainment,UCaO6TYtlC8U5ttz62hTrZgg
1,Stray Kids,UC9rMiEjNaCSsebs31MRDCRA
2,TWICE,UCzgxx_DM2Dcb9Y1spb9mUJA
3,JYP Entertainment,UC14f-9FTI6ZtAisnKaB8wQA
4,JYPAudition,UC4rNDP4pnQabqbmfqcFngHA


In [7]:
jyp = search_df["channelId"][0]
jyp

'UCaO6TYtlC8U5ttz62hTrZgg'

## (1-2) 소속 아티스트 채널 ID 추출

### (1-2-1) 하이브(HYBE)

#### (1번) 뉴진스

In [8]:
#query문 작성
search_response = youtube.search().list(
    q = "NewJeans",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [9]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'GqUm4coFftqtW_7QKYXy6fuRD9A',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 25513, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'Iw-zr96pjnbBNOadk-fZQJPc1ys',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCMki_UkHb4qSc0qyEcOHHJw'},
   'snippet': {'publishedAt': '2022-07-07T08:51:57Z',
    'channelId': 'UCMki_UkHb4qSc0qyEcOHHJw',
    'title': 'NewJeans',
    'description': 'NewJeans Official YouTube NewJeans 공식 유튜브 채널.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/dLGCnr9YbATXl48C6yhJnD-OuEWgTYZIdbLrzJD_YLCR7YtpfChrt-eVfWUsuXWDUDQIwOsO=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/dLGCnr9YbATXl48C6yhJnD-OuEWgTYZIdbLrzJD_YLCR7YtpfChrt-eVfWUsuXWDUDQIwOsO=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/dLGCnr9YbATXl48C6yhJnD-OuEWgTYZIdbLrzJD_YLCR7YtpfChrt-eVfWUsuXWDUDQIwOsO=s800-c-k-c0xffffffff-

In [10]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,NewJeans,UCMki_UkHb4qSc0qyEcOHHJw
1,NewJeans - Topic,UCxOqS3cYg4FaHbobICo7nFQ
2,NewJeans THBN,UCzmMz5wxxPP_MoPBltwZo-w
3,NewJeansのうさぎ,UCfJM4hNfEjsHHqFoZuDFHyg
4,Just For NewJeans,UCpYVgKp4cySG__LpPZU8-2Q


In [42]:
NewJeans = search_df["channelId"][0]
NewJeans

'UCMki_UkHb4qSc0qyEcOHHJw'

#### (2번) 방탄소년단

In [11]:
#query문 작성
search_response = youtube.search().list(
    q = "BANGTANTV",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [12]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'ZCMGRXWC-73l3bt9rqQrtc4-TCk',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 10877, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': '7zUMzrlZNyofQzr63U1cWWE09NY',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCLkAepWjdylmXSltofFvsYQ'},
   'snippet': {'publishedAt': '2012-12-17T03:20:38Z',
    'channelId': 'UCLkAepWjdylmXSltofFvsYQ',
    'title': 'BANGTANTV',
    'description': 'This is the official YouTube channel of BTS. 방탄소년단 공식 유튜브 채널 입니다.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/zL7KDS7WEHVbgpo6dM91hL5e91etHiuq2umW32iSePrhE5411SPsEdpAEcf5m5mYS-a_vpl7Ww=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/zL7KDS7WEHVbgpo6dM91hL5e91etHiuq2umW32iSePrhE5411SPsEdpAEcf5m5mYS-a_vpl7Ww=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/zL7KDS7WEHVbgpo6dM91hL5e91etHiuq2umW32iSePrhE5411SPsEdpAEcf5m5mYS-a_v

In [13]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,BANGTANTV,UCLkAepWjdylmXSltofFvsYQ
1,BANGTANTV MUSIC,UCM_9ANR7oDhIHnp9SF-DVig
2,bangtantv chocolatewingz,UC0HwsVVt-DnFIokD3gUiGLQ
3,BANGTANTV BTS,UCZklKYkhAEMkPWfkHBJxHzw
4,BANGTANTV FARSI,UCz_tOfEn_mSQYHTYZN49efQ


In [46]:
bts = search_df["channelId"][0]
bts

'UCLkAepWjdylmXSltofFvsYQ'

#### (3번) 르세라핌

In [14]:
#query문 작성
search_response = youtube.search().list(
    q = "LE SSERAFIM",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [15]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': '6YceMVQIVJpUctlqBDvbVWg9dp8',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 28769, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'cdqGMDfLkqFciE2Uvm4SEtxsuYA',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCs-QBT4qkj_YiQw1ZntDO3g'},
   'snippet': {'publishedAt': '2021-10-19T08:33:17Z',
    'channelId': 'UCs-QBT4qkj_YiQw1ZntDO3g',
    'title': 'LE SSERAFIM',
    'description': 'LE SSERAFIM Official Youtube 르세라핌 공식 유튜브.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/9CH5aJYgkoM-ZDV-oIF5oQ8uKT3_dMzbTmjMehkPYmyg59CyDaTq7tVib2L5sS6C9srimy2r=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/9CH5aJYgkoM-ZDV-oIF5oQ8uKT3_dMzbTmjMehkPYmyg59CyDaTq7tVib2L5sS6C9srimy2r=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/9CH5aJYgkoM-ZDV-oIF5oQ8uKT3_dMzbTmjMehkPYmyg59CyDaTq7tVib2L5sS6C9srimy2r=s800-c-k-c0xffffffff-n

In [16]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,LE SSERAFIM,UCs-QBT4qkj_YiQw1ZntDO3g
1,LE SSERAFIM - Topic,UC-clMkTZa7k-FxmNgMjoCgQ
2,Lesserafim Mongolia,UC8YaekCQRaHwmNvQT2UX0dA
3,LE SSERAFIM TURKEY,UCkzmUo35LZakaNqnLS2B4Nw
4,Lesserafim Weverse Live - 르세라핌 위버스 라이브,UCWMQD_Uk56zx-wjnRTshgrQ


In [50]:
Lesserafim = search_df["channelId"][0]
Lesserafim

'UCs-QBT4qkj_YiQw1ZntDO3g'

#### (4번) 세븐틴

In [17]:
#query문 작성
search_response = youtube.search().list(
    q = "SEVENTEEN",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [18]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'Br2Kl6MVxlMeyLgPU93gM9uFxW0',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 22084, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'EsG652tk75R6LYUW3-uw3rYrITk',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCfkXDY7vwkcJ8ddFGz8KusA'},
   'snippet': {'publishedAt': '2012-11-16T08:20:35Z',
    'channelId': 'UCfkXDY7vwkcJ8ddFGz8KusA',
    'title': 'SEVENTEEN',
    'description': 'SEVENTEEN Official Youtube #세븐틴 공식 유튜브.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/rZrPk_5wzTppcwthTc9y5TmBWq4GGf-xPtndwWS5T3Y5Cg43uzRTpkoAntBzf-LVgBCYZbG-Yg=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/rZrPk_5wzTppcwthTc9y5TmBWq4GGf-xPtndwWS5T3Y5Cg43uzRTpkoAntBzf-LVgBCYZbG-Yg=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/rZrPk_5wzTppcwthTc9y5TmBWq4GGf-xPtndwWS5T3Y5Cg43uzRTpkoAntBzf-LVgBCYZbG-Yg=s800-c-k-c0xffffffff

In [19]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,SEVENTEEN,UCfkXDY7vwkcJ8ddFGz8KusA
1,SEVENTEEN - Topic,UC0gpUnoyhu44aS3-NxYs7rg
2,Seventeen,UC0IXvMUUPeYNVGe88ieMDEg
3,Seventeen／セブンティーン,UCRILJB5Cy5Qd33h_f61f1Vg
4,Lucky SEVENTEEN,UCxJNpC3iMqIx8VebbRDAOJQ


In [54]:
seventeen = search_df["channelId"][0]
seventeen

'UCfkXDY7vwkcJ8ddFGz8KusA'

#### (5번) 투모로우바이투게더

In [20]:
#query문 작성
search_response = youtube.search().list(
    q = "TOMORROW X TOGETHER OFFICIAL",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [21]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'V_pViVtUsVssbQeVwoqkINp_a7k',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 14016, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'ewGajz5l63TaxBBjEWe7Vmff8-M',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCtiObj3CsEAdNU6ZPWDsddQ'},
   'snippet': {'publishedAt': '2018-12-21T01:41:52Z',
    'channelId': 'UCtiObj3CsEAdNU6ZPWDsddQ',
    'title': 'TOMORROW X TOGETHER OFFICIAL',
    'description': '투모로우바이투게더 공식 유튜브 입니다. This is the official YouTube for #TOMORROW_X_TOGETHER.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/iZQihZKg0dkD89DVBx9iqeUId457HVTLdYojl5hfHauoGYiyVCrfctR64NbD_zjWQPc4T3IX-g=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/iZQihZKg0dkD89DVBx9iqeUId457HVTLdYojl5hfHauoGYiyVCrfctR64NbD_zjWQPc4T3IX-g=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/iZQihZKg0dkD89DVBx9iqeUId457HVTLdYojl5h

In [22]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,TOMORROW X TOGETHER OFFICIAL,UCtiObj3CsEAdNU6ZPWDsddQ
1,TOMORROW X TOGETHER - Topic,UCdeb5b3bAxn1OXXFFeuCaHg
2,TOMORROW X TOGETHER OFFICIAL,UCzCe8j96UWXqKb_uHdE_geg
3,TOMORROW X TOGETHER Official,UCQXnU5NznFEi8S0DcIDb87w
4,TOMORROW X TOGETHER OFFICIAL,UCrbqncDE7zgdCrk2H59l2mw


In [38]:
txt = search_df["channelId"][0]
txt

'UCtiObj3CsEAdNU6ZPWDsddQ'

In [55]:
#유튜브 채널 데이터프레임 생성
channel_hybe = pd.DataFrame()
channel_hybe["channel"] = ["HYBE LABELS","NewJeans","BANGTANTV","LE SSERAFIM","SEVENTEEN","TOMORROW X TOGETHER OFFICIAL"] #채널명
channel_hybe["channel_id"] = [hybe, NewJeans, bts, Lesserafim, seventeen, txt] #채널 id
channel_hybe

,channel,channel_id
0,HYBE LABELS,UC3IZKseVpdzPSBaWxBxundA
1,NewJeans,UCMki_UkHb4qSc0qyEcOHHJw
2,BANGTANTV,UCLkAepWjdylmXSltofFvsYQ
3,LE SSERAFIM,UCs-QBT4qkj_YiQw1ZntDO3g
4,SEVENTEEN,UCfkXDY7vwkcJ8ddFGz8KusA
5,TOMORROW X TOGETHER OFFICIAL,UCtiObj3CsEAdNU6ZPWDsddQ


In [56]:
#channel_hybe를 csv파일로 저장
channel_hybe.to_csv("../../../data/YouTube/channel_hybe.csv", index=False)

### (1-2-2) 와이지(YG)

#### (1번) 블랙핑크

In [23]:
#query문 작성
search_response = youtube.search().list(
    q = "BLACKPINK",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [24]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'K1bMAySUGn3_bl9gvGOZNfknFKQ',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 33802, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'e3zB-ogVH3iCjBYeOA3vaQvfHsI',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCOmHUn--16B90oW2L6FRR3A'},
   'snippet': {'publishedAt': '2016-06-29T03:15:23Z',
    'channelId': 'UCOmHUn--16B90oW2L6FRR3A',
    'title': 'BLACKPINK',
    'description': 'BLACKPINK Official YouTube Channel 블랙핑크 공식 유튜브 채널입니다. JISOO, JENNIE, ROSÉ, LISA 지수, 제니, 로제, 리사.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/hZDUwjoeQqigphL4A1tkg9c6hVp5yXmbboBR7PYFUSFj5PIJSA483NB5v7b0XVoTN9GCku3tqQ=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/hZDUwjoeQqigphL4A1tkg9c6hVp5yXmbboBR7PYFUSFj5PIJSA483NB5v7b0XVoTN9GCku3tqQ=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/hZDUwjoeQqigphL4A1tkg9c6hVp5yXmbboBR7PY

In [25]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,BLACKPINK,UCOmHUn--16B90oW2L6FRR3A
1,BLACKPINK - Topic,UCkbbMCA40i18i7UdjayMPAg
2,BLACKPINK ARCHIVE,UCNO6YcZ_dv7t7fgjsQhly1w
3,BLACKPINK HISPANO,UCmQrJU_kj2gpzr1Cgp1Br5A
4,Blackpink PopCorn,UC-KSjSGzXFHq-EJqGyzKcpA


In [7]:
blackpink = search_df["channelId"][0]
blackpink

'UCOmHUn--16B90oW2L6FRR3A'

#### (2번) 지드래곤

In [26]:
#query문 작성
search_response = youtube.search().list(
    q = "OfficialGDRAGON",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [27]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'jr_Oc7BBeYW03CaeEqf2jFS8mfE',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 10, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': '6tw2NdEK4HgNCfhvYaYCkdu8Sqo',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCeU5qTuBiqTU5z-hyL6WITQ'},
   'snippet': {'publishedAt': '2012-07-17T04:18:48Z',
    'channelId': 'UCeU5qTuBiqTU5z-hyL6WITQ',
    'title': 'OfficialGDRAGON',
    'description': 'G-DRAGON Official YouTube Channel G-DRAGON 공식 유튜브 채널입니다.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AOPolaSnEogRk1BT-Ti9VJ2liGzNkQ_LnzRbdEUTS3db=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/ytc/AOPolaSnEogRk1BT-Ti9VJ2liGzNkQ_LnzRbdEUTS3db=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/ytc/AOPolaSnEogRk1BT-Ti9VJ2liGzNkQ_LnzRbdEUTS3db=s800-c-k-c0xffffffff-no-rj-mo'}},
    'channelTitle': 'OfficialGDRAGON',
    '

In [28]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,OfficialGDRAGON,UCeU5qTuBiqTU5z-hyL6WITQ
1,Official G-DRAGON,UCVAvJqeG9nh-O9hK4xvRQGQ
2,Official GDRAGON,UCXyLacGrQ7z_HMELozu-E2w
3,Official.GDRAGON,UCO-z7YDPGVsHEbqPbq3sqEg
4,jiyonglanes jiyong,UCMWaE07EzGVM5ZTzZ3h-1ew


In [11]:
gd = search_df["channelId"][0]
gd

'UCeU5qTuBiqTU5z-hyL6WITQ'

#### (3번) 트레저

In [32]:
#query문 작성
search_response = youtube.search().list(
    q = "TREASURE (트레저)",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [33]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'JpLR51lZiKkemwbqBAtIuCutMcc',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 13581, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'QXNX9naSulPIz9QygedE7NJYdE4',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCx9hXYOCvUYwrprEqe4ZQHA'},
   'snippet': {'publishedAt': '2019-02-08T02:21:20Z',
    'channelId': 'UCx9hXYOCvUYwrprEqe4ZQHA',
    'title': 'TREASURE (트레저)',
    'description': 'TREASURE Official YouTube Channel 트레저 공식 유튜브 채널입니다. CHOI HYUN SUK, JIHOON, YOSHI, JUNKYU, YOON JAE ...',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/yUHNt_Dre7041FVc1PYgQJddgOIdD5XGes7Sv5pilNcdZ2uqq_Th5xe4Ymm4P9oYpEb3cVAZ6cU=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/yUHNt_Dre7041FVc1PYgQJddgOIdD5XGes7Sv5pilNcdZ2uqq_Th5xe4Ymm4P9oYpEb3cVAZ6cU=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/yUHNt_Dre7041FVc1PYgQJddg

In [34]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,TREASURE (트레저),UCx9hXYOCvUYwrprEqe4ZQHA
1,TREASURE - Topic,UCe5f_XquYL3uB-A5mdy-5-A
2,YG TREASURE BOX,UCquCOGattLleH5erOxiWVmQ
3,everYthinG 애명,UC6vDtl2QNospNu3e-fxp2Ag
4,쟈분젤 Jabunjel,UCeaqzcshmz2lMi42bKxXbGQ


In [15]:
treasure = search_df["channelId"][0]
treasure

'UCx9hXYOCvUYwrprEqe4ZQHA'

#### (4번) 위너

In [35]:
#query문 작성
search_response = youtube.search().list(
    q = "WINNER",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [36]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'IQImvHOnQNhFhs-gAyOfXTqTPEA',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 35877, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'eY4cIsr5FV2TlhOLRRd6DEfUev0',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCayQxFAoFCvGariuQCtHRGQ'},
   'snippet': {'publishedAt': '2013-12-10T02:42:56Z',
    'channelId': 'UCayQxFAoFCvGariuQCtHRGQ',
    'title': 'WINNER',
    'description': 'WINNER Official YouTube Channel 위너 공식 유튜브 채널입니다. YOON, JINU, HOONY, MINO 강승윤, 김진우, 이승훈, 송민호.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AOPolaQZYsenI7L4IZNvhR2dnrVRgCMqbG9BeYc2PMzVgw=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/ytc/AOPolaQZYsenI7L4IZNvhR2dnrVRgCMqbG9BeYc2PMzVgw=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/ytc/AOPolaQZYsenI7L4IZNvhR2dnrVRgCMqbG9BeYc2PMzVgw=s800-c-k-c0xffffffff-no-rj-mo'}},
    'cha

In [37]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,WINNER,UCayQxFAoFCvGariuQCtHRGQ
1,WINNER - Topic,UCYFMusbxIBk6gosFW0o882A
2,Winner,UC_QjJqfnLkrEmaZnWDOMskw
3,Winner(위너),UCZ1pHpliTMw1-ffpP2Qalhg
4,Winners Team,UC1vqsWUiZpV3-_D4zVutaMQ


In [19]:
winner = search_df["channelId"][0]
winner

'UCayQxFAoFCvGariuQCtHRGQ'

In [24]:
#유튜브 채널 데이터프레임 생성
channel_yg = pd.DataFrame()
channel_yg["channel"] = ["YG ENTERTAINMENT","BLACKPINK","OfficialGDRAGON","TREASURE (트레저)","WINNER"] #채널명
channel_yg["channel_id"] = [yg, blackpink, gd, treasure, winner] #채널 id
channel_yg

,channel,channel_id
0,YG ENTERTAINMENT,UCQi67q4kGdmnJaRzX81uK5g
1,BLACKPINK,UCOmHUn--16B90oW2L6FRR3A
2,OfficialGDRAGON,UCeU5qTuBiqTU5z-hyL6WITQ
3,TREASURE (트레저),UCx9hXYOCvUYwrprEqe4ZQHA
4,WINNER,UCayQxFAoFCvGariuQCtHRGQ


In [25]:
#channel_yg를 csv파일로 저장
channel_yg.to_csv("../../../data/YouTube/channel_yg.csv", index=False)

### (1-2-3) 제이와이피(JYP)

#### (1번) 스트레이키즈

In [38]:
#query문 작성
search_response = youtube.search().list(
    q = "Stray Kids",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [39]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': '2eQr6T8gl_nf4zaSDp2sA4dwHlo',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 24290, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'n-qjoWj5sZXhyDazbEmfuPiQEG0',
   'id': {'kind': 'youtube#channel', 'channelId': 'UC9rMiEjNaCSsebs31MRDCRA'},
   'snippet': {'publishedAt': '2017-05-12T03:28:10Z',
    'channelId': 'UC9rMiEjNaCSsebs31MRDCRA',
    'title': 'Stray Kids',
    'description': 'Stray Kids Official YouTube.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/nO0HX2YwTdh1YDCuxRsCd9ljmK2cbUVFu7PGoT22LPKx4Zk0iqhUJbheybIpC-0hqEUIIC0dS3g=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/nO0HX2YwTdh1YDCuxRsCd9ljmK2cbUVFu7PGoT22LPKx4Zk0iqhUJbheybIpC-0hqEUIIC0dS3g=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/nO0HX2YwTdh1YDCuxRsCd9ljmK2cbUVFu7PGoT22LPKx4Zk0iqhUJbheybIpC-0hqEUIIC0dS3g=s800-c-k-c0xffffffff-no-rj-

In [40]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,Stray Kids,UC9rMiEjNaCSsebs31MRDCRA
1,Stray Kids - Topic,UCIMmuidNJdncfMEelOU08Fg
2,Stray Kids Japan Official YouTube,UCXhj2pPWvONXmvgHX5wllCA
3,STRAY KIDS DISTRICT,UC_iFLJd2mAvf1VFhpmU9iqA
4,Stray Kids Vlive Archives,UCqKkkyTu4DvUq44APQx2n2A


In [11]:
straykids = search_df["channelId"][0]
straykids

'UC9rMiEjNaCSsebs31MRDCRA'

#### (2번) 트와이스

In [41]:
#query문 작성
search_response = youtube.search().list(
    q = "TWICE",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [42]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'JR5YtkVRhLusgEVLRd7wVO1wxxI',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 36854, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'iubCVmIbbMBuu6zypHV2rYS2E6U',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCzgxx_DM2Dcb9Y1spb9mUJA'},
   'snippet': {'publishedAt': '2016-03-27T15:48:08Z',
    'channelId': 'UCzgxx_DM2Dcb9Y1spb9mUJA',
    'title': 'TWICE',
    'description': 'TWICE Official YouTube Channel.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/PKCD8EPU-3nm1UQ54-e2ZT3Xqwr9cMA2F-vow1gnkFGCxzwv5hRZgbjBssLe4ttjhwpXXgDbdg=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/PKCD8EPU-3nm1UQ54-e2ZT3Xqwr9cMA2F-vow1gnkFGCxzwv5hRZgbjBssLe4ttjhwpXXgDbdg=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/PKCD8EPU-3nm1UQ54-e2ZT3Xqwr9cMA2F-vow1gnkFGCxzwv5hRZgbjBssLe4ttjhwpXXgDbdg=s800-c-k-c0xffffffff-no-rj-mo'}}

In [43]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,TWICE,UCzgxx_DM2Dcb9Y1spb9mUJA
1,TWICE - Topic,UCAq0pFGa2w9SjxOq0ZxKVIw
2,TWICE JAPAN OFFICIAL YouTube Channel,UCCRb6nYKaT8tzLA8CwDdUtw
3,Twice,UCNOCNM_WCTVuwSUYvj5SEkw
4,Arab Twice Team,UC0EakU8Gklv7X_6G70DrJVg


In [15]:
twice = search_df["channelId"][0]
twice

'UCzgxx_DM2Dcb9Y1spb9mUJA'

#### (3번) 엔믹스

In [44]:
#query문 작성
search_response = youtube.search().list(
    q = "NMIXX",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [45]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'CvzlHR_Y4xUX5b7gZUXBGghJxnE',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 23143, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'Zlgo1svatrZPQdXCSAxfyIhFJD0',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCnUAyD4t2LkvW68YrDh7fDg'},
   'snippet': {'publishedAt': '2021-07-12T03:58:53Z',
    'channelId': 'UCnUAyD4t2LkvW68YrDh7fDg',
    'title': 'NMIXX',
    'description': 'NMIXX Official YouTube.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/OZj8QeoGfH-S1YoZmZs7Ymr6JKmlaDfgPgAGg8XGNZSbbRIXP6BfXyyTrWZu99bgafNRKaMv7Q=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/OZj8QeoGfH-S1YoZmZs7Ymr6JKmlaDfgPgAGg8XGNZSbbRIXP6BfXyyTrWZu99bgafNRKaMv7Q=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/OZj8QeoGfH-S1YoZmZs7Ymr6JKmlaDfgPgAGg8XGNZSbbRIXP6BfXyyTrWZu99bgafNRKaMv7Q=s800-c-k-c0xffffffff-no-rj-mo'}},
    'c

In [46]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,NMIXX,UCnUAyD4t2LkvW68YrDh7fDg
1,NMIXX - Topic,UC_Cx288SDUD9liYn7CiJLAA
2,NMIXX Turkey,UCTRvSnt4PTR9azum9UBzdXg
3,NMIXX SQUAD,UCmjcXc2gTZjwxCreZc_JTBQ
4,NMIXX Vlive Archive 엔믹스 브이라이브 아카이브,UCXtpDAe0l2UsHU4ksFL15hg


In [19]:
nmixx = search_df["channelId"][0]
nmixx

'UCnUAyD4t2LkvW68YrDh7fDg'

#### (4번) 데이식스

In [47]:
#query문 작성
search_response = youtube.search().list(
    q = "DAY6",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [48]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': 'uiEZWcsG4f7Wcx92q8WLXbpuvIo',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 28038, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': '3twlN2VnOtDU2G-BvRelReFF6NY',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCFEVXX0YX4eT6tdNkXi40vg'},
   'snippet': {'publishedAt': '2015-09-18T06:51:55Z',
    'channelId': 'UCFEVXX0YX4eT6tdNkXi40vg',
    'title': 'DAY6 - Topic',
    'description': '',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/wzZVAOm3ilLgi4vZC4Ww6G9Y4bv_1z2dsRsL5roR2HwcO2qAYCszJk29e2C0dpxdKHM9i031Ag=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/wzZVAOm3ilLgi4vZC4Ww6G9Y4bv_1z2dsRsL5roR2HwcO2qAYCszJk29e2C0dpxdKHM9i031Ag=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/wzZVAOm3ilLgi4vZC4Ww6G9Y4bv_1z2dsRsL5roR2HwcO2qAYCszJk29e2C0dpxdKHM9i031Ag=s800-c-k-c0xffffffff-no-rj-mo'}},
    'channelTitle': 'D

In [49]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,DAY6 - Topic,UCFEVXX0YX4eT6tdNkXi40vg
1,DAY6,UCp-pqXsizklX3ZHvLxXyhxw
2,WholeWeek Day6,UC-NhNhCHIXv1Pmyqo1u98Mw
3,Young K (Day6) - Topic,UCDqOf2y4k-y8YGecDlPjNOQ
4,DAY6​ (for SUNGJIN)​ VLIVE​ Archive​ with SUBs,UCklirTVKFFsK1ldnAj5bqnw


In [23]:
day6 = search_df["channelId"][1]
day6

'UCp-pqXsizklX3ZHvLxXyhxw'

#### (5번) 있지

In [50]:
#query문 작성
search_response = youtube.search().list(
    q = "ITZY",
    order = "relevance",
    part = "snippet",
    maxResults = 5,
    type = "channel"
).execute()

In [51]:
search_response

{'kind': 'youtube#searchListResponse',
 'etag': '25By0CAOLptoX6fdiqXcB_zrTok',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'KR',
 'pageInfo': {'totalResults': 25863, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'ojRLkrmLO8HtGC1TmRjoDFt4kE4',
   'id': {'kind': 'youtube#channel', 'channelId': 'UCDhM2k2Cua-JdobAh5moMFg'},
   'snippet': {'publishedAt': '2019-01-14T08:52:03Z',
    'channelId': 'UCDhM2k2Cua-JdobAh5moMFg',
    'title': 'ITZY',
    'description': 'ITZY "KILL MY DOUBT" ▽▽▽CHECK OUT LINKS.',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/5qSAjhri_C54rgk26T_vMlTOU4OcIdgOv4hxMwnfs_t8N09Mpp-cT7bgbImNxoTf_P1xQDN-bg=s88-c-k-c0xffffffff-no-rj-mo'},
     'medium': {'url': 'https://yt3.ggpht.com/5qSAjhri_C54rgk26T_vMlTOU4OcIdgOv4hxMwnfs_t8N09Mpp-cT7bgbImNxoTf_P1xQDN-bg=s240-c-k-c0xffffffff-no-rj-mo'},
     'high': {'url': 'https://yt3.ggpht.com/5qSAjhri_C54rgk26T_vMlTOU4OcIdgOv4hxMwnfs_t8N09Mpp-cT7bgbImNxoTf_P1xQDN-bg=s800-c-k-c0xffffffff-no-

In [52]:
#필요한 변수들로 데이터프레임을 구성
channelTitle = []
channelId = []

for item in search_response["items"]:
    channelTitle.append(item["snippet"]["title"])
    channelId.append(item["snippet"]["channelId"])
    
search_df = pd.DataFrame({"channelTitle": channelTitle, "channelId": channelId})
search_df

,channelTitle,channelId
0,ITZY,UCDhM2k2Cua-JdobAh5moMFg
1,ITZY - Topic,UCTP45_DE3fMLujU8sZ-MBzw
2,ITZY JAPAN OFFICIAL YouTube Channel,UCJsfO4nKS3_c24zLbF8rT0A
3,itzy zone,UC9kyC1BZizd15dh1tPLjw2g
4,Itzy’s_loco,UCOgmLtgsNaInBAikLisSIHg


In [27]:
itzy = search_df["channelId"][0]
itzy

'UCDhM2k2Cua-JdobAh5moMFg'

In [28]:
#유튜브 채널 데이터프레임 생성
channel_jyp = pd.DataFrame()
channel_jyp["channel"] = ["JYP Entertainment","Stray Kids","TWICE","NMIXX","DAY6","ITZY"] #채널명
channel_jyp["channel_id"] = [jyp, straykids, twice, nmixx, day6, itzy] #채널 id
channel_jyp

,channel,channel_id
0,JYP Entertainment,UCaO6TYtlC8U5ttz62hTrZgg
1,Stray Kids,UC9rMiEjNaCSsebs31MRDCRA
2,TWICE,UCzgxx_DM2Dcb9Y1spb9mUJA
3,NMIXX,UCnUAyD4t2LkvW68YrDh7fDg
4,DAY6,UCp-pqXsizklX3ZHvLxXyhxw
5,ITZY,UCDhM2k2Cua-JdobAh5moMFg


In [29]:
#channel_jyp를 csv파일로 저장
channel_jyp.to_csv("../../../data/YouTube/channel_jyp.csv", index=False)